In [5]:
dataset_train = "dataset\\ingatlan.com_training_final.csv"
dataset_val = "dataset\\ingatlan.com_testing_final.csv"
dataset = "dataset\\ingatlan.com_full_final.csv"
RANDOM_STATE = 39612
import numpy as np
import matplotlib as plt
import pandas as pd
import re
import roman
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

In [7]:
mldf = pd.read_csv('finalized_for_train.csv')
mldf

FileNotFoundError: [Errno 2] No such file or directory: 'finalized_for_train.csv'

In [ ]:
def train_eval_linreg(df,target_cols):
    X = df[target_cols]
    y = df['price_created_at']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

    model = LinearRegression()
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'LR Mean Squared Error: {mse}')
    mape = mean_absolute_percentage_error(y_test, y_pred)*100
    print(f'LR Mean Absolute Percentage Error: {mape}')

def train_eval_gbm_gridsearch(df,target_cols):
    X = df[target_cols]
    y = df['price_created_at']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 5]
    }

    gbm = GradientBoostingRegressor(random_state=RANDOM_STATE)
    grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_percentage_error', n_jobs=-1)

    # Fit GridSearchCV
    grid_search.fit(X_train, y_train)
    best_gbm = grid_search.best_estimator_
    best_gbm.fit(X_train, y_train)

    y_pred = best_gbm.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f"GBM Mean Squared Error: {mse}")
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    print(f"GBM Mean Absolute Percentage Error: {mape}")
    print(f"Best GBM params {best_gbm.get_params}")

def train_eval_gbm(df,target_cols):
    X = df[target_cols]
    y = df['price_created_at']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

    gbm = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=5, random_state=RANDOM_STATE)

    gbm.fit(X_train, y_train)
    y_pred = gbm.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"GBM Mean Squared Error: {mse}")
    mape = mean_absolute_percentage_error(y_test, y_pred)*100
    print(f"GBM Mean Absolute Percentage Error: {mape}")
    #return(mse,mape)


#target_cols = ['property_area', 'room_cnt', 'small_room_cnt', 'balcony_area', 'elevator_type',
#               'city','property_subtype','garden_access','heating_type','orientation',
#               'property_floor', 'building_floor_count', 'property_condition_type',
#               'created_at']


---

> Tanítási start

In [ ]:
mldf['meroszam'] = (mldf['ad_view_cnt'] / mldf['active_days'])*100

In [ ]:
target_cols = mldf.columns
target_cols = target_cols.drop('meroszam')
#target_cols = target_cols.drop('active_days')
#target_cols = target_cols.drop('ad_view_cnt')
target_cols = target_cols.drop('price_created_at')

train_eval_linreg(mldf,target_cols)
train_eval_gbm_gridsearch(mldf,target_cols)
#train_eval_gbm(mldf,target_cols)

LR Mean Squared Error: 32.78822098852316
LR Mean Absolute Percentage Error: 24.087435831977263
GBM Mean Squared Error: 11.191825981056413
GBM Mean Absolute Percentage Error: 11.70590575584258
Best GBM params <bound method BaseEstimator.get_params of GradientBoostingRegressor(max_depth=5, n_estimators=300, random_state=39612)>


In [ ]:
def train_eval_xgbm(df, target_cols):
    X = df[target_cols.drop('price_created_at')]
    y = df['price_created_at']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

    xgbm = XGBRegressor(n_estimators=650, learning_rate=0.07, max_depth=13, random_state=RANDOM_STATE, n_jobs=-1)
    xgbm.fit(X_train, y_train)
    y_pred = xgbm.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f"XGBM Mean Squared Error: {mse}")
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    print(f"XGBM Mean Absolute Percentage Error: {mape}")

train_eval_xgbm(mldf, target_cols)

NameError: name 'mldf' is not defined